In [1]:
import pandas as pd
import os
import numpy as np
from ibm_watson_machine_learning.helpers.connections import DataConnection,CP4DAssetLocation,AssetLocation
from ibm_watson_machine_learning.experiment import AutoAI
from ibm_watson_machine_learning import APIClient
import project_lib
from tqdm import tqdm
tqdm.pandas()
project = project_lib.Project()
%matplotlib inline

#### Note: Insert user name and password

In [2]:
token = os.environ['USER_ACCESS_TOKEN']
wml_credentials = {
        "token": token,
        "instance_id" : "openshift",
        "url": os.environ['RUNTIME_ENV_APSX_URL'],
        "version": "3.5",
        "username": "insert_username", #e.g. mluser1
        "password": "insert_password" #e.g. xxxxxxx
    
     }

client = APIClient(wml_credentials)
client.set.default_project(os.environ['PROJECT_ID'])

'SUCCESS'

In [3]:
# Specify a name for the target deployment space
SPACE_NAME = 'ClaimsLeakage'

# Check is the space already exists
space_uid = ''
for space in client.spaces.get_details()['resources']:
    if space['entity']['name'] == SPACE_NAME:
        space_uid=space['metadata']['id']
        client.set.default_space(space_uid)
        break

if space_uid == '':
    print("Deployment space",SPACE_NAME,"not found.")

Unsetting the project_id ...


In [20]:
df_predictions_example = pd.read_csv(project.get_file('regression_leakage_autoAI.csv'))
Handler_df = pd.read_csv('/project_data/data_asset/current_assignments.csv')

In [96]:
features_to_use = ['Base Claims Cost',
 'Handler Case Load at Allocation',
 'Notification Delay',
 'Claim Complexity',
 'Claim Type',
 'Days from Policy Inception to Date of Claim',
 'Claim Handler Level',
 'Number of Previous Claims',
 'Claim Handler Minus Complexity']

In [97]:
Handler_df

Agent  Experience level  Claims    Leakage  Average Leakage  \
0    ClaimHandler3               1.0   430.0  126337.00       293.806977   
1   ClaimHandler12               1.0  1315.0  109727.80        83.443194   
2   ClaimHandler15               1.0  1360.0  112071.00        82.405147   
3    ClaimHandler7               1.0  1210.0   96555.00        79.797521   
4   ClaimHandler16               2.0   690.0   67675.80        98.080870   
5   ClaimHandler14               2.0  1050.0  100311.10        95.534381   
6    ClaimHandler6               2.0  1165.0   96356.80        82.709700   
7   ClaimHandler20               2.0  1040.0   73818.80        70.979615   
8    ClaimHandler9               2.0  1135.0   74904.20        65.994890   
9    ClaimHandler2               2.0  1050.0   67399.40        64.189905   
10  ClaimHandler11               2.0  1480.0   66934.30        45.225878   
11  ClaimHandler17               3.0  1070.0   91211.70        85.244579   
12  ClaimHandler13               3.0  1510.0   78279.30        51.840596   
13  ClaimHandler10               3.0  1780.0   83662.10        47.001180   
14   ClaimHandler8               3.0  1615.0   68342.10        42.317090   
15   ClaimHandler5               3.0  1770.0   47073.10        26.594972   
16   ClaimHandler4               3.0  1970.0   49772.50        25.265228   
17  ClaimHandler18               4.0   870.0   27679.15        31.815115   
18   ClaimHandler1               4.0   745.0   19147.30        25.701074   
19  ClaimHandler19               4.0  1800.0   36715.80        20.397667   

    Average claim complexity  Total claim amount  
0                        2.4       185738.472298  
1                        1.9       140155.310887  
2                        1.7       139876.245487  
3                        1.4       120456.320484  
4                        3.5        84193.173749  
5                        3.4       124691.637213  
6                        2.5       119157.143471  
7                        3.3        89273.346844  
8                        2.9        90102.992451  
9                        3.2        80273.395259  
10                       2.6        78787.138399  
11                       4.3       107232.772579  
12                       4.3        91571.161654  
13                       4.1        96189.902334  
14                       4.0        77942.935421  
15                       4.0        52027.946098  
16                       3.6        53900.489347  
17                       5.0        29712.320890  
18                       4.8        20257.278225  
19                       4.8        36873.357809

In [100]:
df = pd.read_csv('/project_data/data_asset/new_claim_cases.csv',index_col='CLAIM_NBR',usecols=features_to_use+['CLAIM_NBR'])

In [101]:
df.columns.to_list()

['Base Claims Cost',
 'Handler Case Load at Allocation',
 'Notification Delay',
 'Claim Complexity',
 'Claim Type',
 'Days from Policy Inception to Date of Claim',
 'Claim Handler Level',
 'Number of Previous Claims',
 'Claim Handler Minus Complexity']

In [102]:
client.deployments.list()

------------------------------------  ---------------------  -----  ------------------------
GUID                                  NAME                   STATE  CREATED
57966280-43d8-4303-894b-2f1bd099250e  Pipeline_7_webservice  ready  2021-04-20T08:27:28.044Z
------------------------------------  ---------------------  -----  ------------------------


In [114]:
def score_claim(claim,handler_df):
    to_score = []
    for k,each in handler_df.iterrows():
        np_claim = claim.copy()
        np_claim['Handler Allocation'] = each['Agent']
        np_claim['Handler Case Load at Allocation'] = each['Claims']
        np_claim['Claim Handler Level'] = each['Experience level']
        to_score.append(np_claim.values)
        cols = np_claim.index.tolist()
        del np_claim
    to_score = pd.DataFrame(to_score,columns = cols)
    return to_score

In [115]:
grouped = df.reset_index().progress_apply(lambda x: score_claim(x,Handler_df),axis=1).values

#### Note: Insert GUID of Pipeline_7_webservice from the client.deployments.list() print out above

In [116]:
results = []

for each in grouped:
    payload = {"input_data":[{
                    "fields": each.drop(columns=['CLAIM_NBR','Handler Allocation']).columns.tolist(),
                    "values": each.drop(columns=['CLAIM_NBR','Handler Allocation']).values.tolist()
            }]}
    results.extend(client.deployments.score('insert_GUID',payload)['predictions'][0]['values'])

In [117]:
results_df = pd.concat([each for each in grouped])

In [118]:
results_df['Pred_Leakage'] = list(map(lambda x:x[0],results))

In [119]:
results_df

CLAIM_NBR  Base Claims Cost  Handler Case Load at Allocation  \
0    MA73673       1471.983592                            430.0   
1    MA73673       1471.983592                           1315.0   
2    MA73673       1471.983592                           1360.0   
3    MA73673       1471.983592                           1210.0   
4    MA73673       1471.983592                            690.0   
..       ...               ...                              ...   
15   MA90658        675.568127                           1770.0   
16   MA90658        675.568127                           1970.0   
17   MA90658        675.568127                            870.0   
18   MA90658        675.568127                            745.0   
19   MA90658        675.568127                           1800.0   

    Notification Delay  Claim Complexity         Claim Type  \
0                 22.0               3.0   Personal Effects   
1                 22.0               3.0   Personal Effects   
2                 22.0               3.0   Personal Effects   
3                 22.0               3.0   Personal Effects   
4                 22.0               3.0   Personal Effects   
..                 ...               ...                ...   
15                21.0               2.0  Accidental Damage   
16                21.0               2.0  Accidental Damage   
17                21.0               2.0  Accidental Damage   
18                21.0               2.0  Accidental Damage   
19                21.0               2.0  Accidental Damage   

    Days from Policy Inception to Date of Claim  Claim Handler Level  \
0                                         274.0                  1.0   
1                                         274.0                  1.0   
2                                         274.0                  1.0   
3                                         274.0                  1.0   
4                                         274.0                  2.0   
..                                          ...                  ...   
15                                        226.0                  3.0   
16                                        226.0                  3.0   
17                                        226.0                  4.0   
18                                        226.0                  4.0   
19                                        226.0                  4.0   

    Number of Previous Claims  Claim Handler Minus Complexity  \
0                         1.0                             0.0   
1                         1.0                             0.0   
2                         1.0                             0.0   
3                         1.0                             0.0   
4                         1.0                             0.0   
..                        ...                             ...   
15                        1.0                             2.0   
16                        1.0                             2.0   
17                        1.0                             2.0   
18                        1.0                             2.0   
19                        1.0                             2.0   

   Handler Allocation  Pred_Leakage  
0       ClaimHandler3      0.169280  
1      ClaimHandler12      0.352303  
2      ClaimHandler15      0.352303  
3       ClaimHandler7      0.352303  
4      ClaimHandler16      0.197365  
..                ...           ...  
15      ClaimHandler5      0.442785  
16      ClaimHandler4      0.442785  
17     ClaimHandler18      0.248304  
18      ClaimHandler1      0.138332  
19     ClaimHandler19      0.413549  

[2000 rows x 12 columns]

In [120]:
results_df

CLAIM_NBR  Base Claims Cost  Handler Case Load at Allocation  \
0    MA73673       1471.983592                            430.0   
1    MA73673       1471.983592                           1315.0   
2    MA73673       1471.983592                           1360.0   
3    MA73673       1471.983592                           1210.0   
4    MA73673       1471.983592                            690.0   
..       ...               ...                              ...   
15   MA90658        675.568127                           1770.0   
16   MA90658        675.568127                           1970.0   
17   MA90658        675.568127                            870.0   
18   MA90658        675.568127                            745.0   
19   MA90658        675.568127                           1800.0   

    Notification Delay  Claim Complexity         Claim Type  \
0                 22.0               3.0   Personal Effects   
1                 22.0               3.0   Personal Effects   
2                 22.0               3.0   Personal Effects   
3                 22.0               3.0   Personal Effects   
4                 22.0               3.0   Personal Effects   
..                 ...               ...                ...   
15                21.0               2.0  Accidental Damage   
16                21.0               2.0  Accidental Damage   
17                21.0               2.0  Accidental Damage   
18                21.0               2.0  Accidental Damage   
19                21.0               2.0  Accidental Damage   

    Days from Policy Inception to Date of Claim  Claim Handler Level  \
0                                         274.0                  1.0   
1                                         274.0                  1.0   
2                                         274.0                  1.0   
3                                         274.0                  1.0   
4                                         274.0                  2.0   
..                                          ...                  ...   
15                                        226.0                  3.0   
16                                        226.0                  3.0   
17                                        226.0                  4.0   
18                                        226.0                  4.0   
19                                        226.0                  4.0   

    Number of Previous Claims  Claim Handler Minus Complexity  \
0                         1.0                             0.0   
1                         1.0                             0.0   
2                         1.0                             0.0   
3                         1.0                             0.0   
4                         1.0                             0.0   
..                        ...                             ...   
15                        1.0                             2.0   
16                        1.0                             2.0   
17                        1.0                             2.0   
18                        1.0                             2.0   
19                        1.0                             2.0   

   Handler Allocation  Pred_Leakage  
0       ClaimHandler3      0.169280  
1      ClaimHandler12      0.352303  
2      ClaimHandler15      0.352303  
3       ClaimHandler7      0.352303  
4      ClaimHandler16      0.197365  
..                ...           ...  
15      ClaimHandler5      0.442785  
16      ClaimHandler4      0.442785  
17     ClaimHandler18      0.248304  
18      ClaimHandler1      0.138332  
19     ClaimHandler19      0.413549  

[2000 rows x 12 columns]

In [121]:
results_df.to_csv('/project_data/data_asset/predictions_for_DO_production.csv',index=False)